# Title: Binary classification of subscriptions for direct marketing campaigns: A logistic regression & random forest approach
### Author: Georgios Spyrou (georgios.spyrou1@gmail.com)
### Completion Date: 04/08/2020

## Project Description

Purpose of this project is to analyze a dataset containing information about marketing campaigns that were conducted via phone calls from a Portuguese banking institution to their clients. Main goal of these campaigns was to attempt and persuade their clients to subscribe to a specific product (bank term deposit). At the end of each call the client was either ended up subscibing to the product (successful campaing) or not (unsuccessful campaing). The dataset contains 21 features (columns) and 41188 instances of calls to clients (rows). Please note that there are cases where the same client was contacted multiple times - something that practically doesn't affect our analysis as each call is being considered indipendent from another. If a client has been contacted multiple times is being captured in the features, as presented below:

### Variables

The predictor variables (features) contained in the dataset and that we will use in order to predict the probability that a client will subscribe for the product, can be divided into 5 sections:

1. Features describing the bank client:

    a. **age** <br>
    b. **job**: type of job <br>
    c. **marital**: marital status <br>
    d. **education**: education level <br>
    e. **default**: has credit in default? <br>
    f. **housing**: has housing loan? <br>
    g. **loan**: has personal loan?<br>
    <br>
2. Features related to the last contact of the current campaign:
    a. **contact**: contact communication type <br>
    b. **month**: last contact month of year <br>
    c. **day_of_week**: last contact day of the week <br>
    d. **duration**: last contact duration, in seconds <br>
    <br>
3. Other features related to the campaign(s):
    a. **campaign**: # of contacts performed during this campaign and for this client <br>
    b. **pdays**: # of days that passed by after the client was last contacted from a previous campaign <br>
    c. **previous**: # of contacts performed before this campaign and for this client <br>
    d. **poutcome**: outcome of the previous marketing campaign <br>
    <br>   
4. Social and Economic features:
    a. **emp.var.rate**: employement variation rate - quarterly indicator <br>
    b. **cons.price.idx**: consumer price index - monthly indicator <br>
    c. **cons.conf.idx**: consumer confidence index - monthly indicator <br>
    d. **euribor3m**: euribor 3 month rate - daily indicator <br>
    e. **nr.employed**: # of employees - quarterly indicator <br>
      
Of course we can also observe the variable **subscribed** which is the target variable for our project, indicating if the client subscribed to the product ('yes') or not ('no').

### Part 1 - Data import and cleaning